# `click`

## 1. Introduction
Click is a Python package for creating beautiful, composable command-line interfaces with minimal boilerplate. It uses decorators to define commands, options, and arguments, and handles:

* Parsing and validation
* Automatic --help generation
* Rich types (files, paths, choices)
* Nested subcommands via groups
* Prompting, confirmation, and progress bars

In SafeChess, we use Click to build our safechess analyze CLI.

## 2. Paradigm & Key Objects

|Concept|	What It Does|
|---|---|
|`@click.command()`|	Marks a function as a standalone CLI command|
|`@click.group()`	|Defines a command group (for subcommands)|
|`@click.option()`|	Declares a flag or option (`--depth 12`)|
|`@click.argument()`|	Declares a positional argument (`input.pgn`)|
|Built-in types|	`click.Path`, `click.File`, `click.Choice`, etc.|
|Output & I/O|	`click.echo()`, `click.secho()`|
|Context|	`click.Context` for shared state across commands|
|Error handling|	`click.BadParameter`, `click.UsageError`, `click.Abort`|

All decorators modify your function signature so that when you call the script, Click handles parsing and passes validated values into your function.

## 3. Key Functionality

### Installation

```bash
pip install click
```

### Simple Command

In [ ]:
import click

@click.command()                     # 1. Declare this function as a CLI
@click.option("--count", "-c",
              default=1,
              help="Number of greetings")
@click.argument("name")              # 2. One positional argument
def greet(count, name):
    """Print HELLO <name> COUNT times."""
    for _ in range(count):
        click.echo(f"Hello, {name}!")

if __name__ == "__main__":
    greet()                          # 3. Entry point


Run: python greet.py Alice → prints Hello, Alice!

--help auto-generated: python greet.py --help

### Options vs Arguments

* Options start with `-`/`--`, can be required/optional, have types, defaults, and help texts.
* Arguments are positional, required by default, and validated by `Click` types.

In [ ]:
@click.option("--depth", type=int, default=12, help="Search depth")
@click.argument("pgn", type=click.Path(exists=True))

### File and Path type

In [ ]:
@click.option("--output", 
              type=click.File("w"),   # open for writing
              default="results.json")
def save(output):
    output.write('{"ok":true}')

* `click.Path(exists=True)` -> ensures the given path exists (or not, if exists=False).
* `click.File("r"/"w")` -> opens the file for you, passes the file object.

### Command groups and subcommands

In [ ]:
@click.group()        # root group
def cli():
    """SafeChess CLI."""

@cli.command()        # subcommand: analyze
@click.option("--depth", type=int, default=15)
@click.argument("pgn", type=click.Path(exists=True))
def analyze(depth, pgn):
    """Analyze a PGN file."""
    click.echo(f"Analyzing {pgn} at depth {depth}")

@cli.command()        # another subcommand: export
@click.argument("db", type=click.Path(exists=True))
def export(db):
    """Export results from DB."""
    click.echo(f"Exporting from {db}")

if __name__ == "__main__":
    cli()

Usage examples:
```bash
safechess analyze games.pgn

safechess export results.db
```

### Prompting and Confirmation

In [ ]:
@click.command()
@click.option("--force", is_flag=True,
              help="Skip confirmation")
def delete(force):
    if not force:
        click.confirm("Really delete all data?", abort=True)
    click.echo("Deleted!")

`click.confirm(..., abort=True)` -> exits with an error if user says “no”.

### Custom Validation and Errors

In [ ]:
@click.command()
@click.option("--rate", type=float)
def set_rate(rate):
    if rate <= 0 or rate > 1:
        raise click.BadParameter("rate must be between 0 and 1")
    click.echo(f"Rate set to {rate}")

Raising `BadParameter` shows the option’s usage and your message.

### Testing with CLIRunner

In [ ]:
from click.testing import CliRunner
from mycli import analyze

def test_analyze_minimal(tmp_path):
    pgn = tmp_path / "g.pgn"
    pgn.write_text("1. e4 e5")
    runner = CliRunner()
    result = runner.invoke(analyze, ["--depth", "5", str(pgn)])
    assert result.exit_code == 0
    assert "Analyzing" in result.output

`CliRunner().invoke()` simulates running the command and captures output & exit codes.